# Biking Script

All python scripts combined into one for ease of testing and manipulating. Some functions renamed to avoid conflicts, these are noted under the subtitles.

June 11, 2020 status:
* Update notebook AND scripts so that selecting no manipulation from GUI will result in keeping both time and date columns. Note that these columns must be taken out for MatLap entropy script to work.

# start_biking.py

## start( )

NOTE: 

* GUI libraries not imported
* Custom python library (dynamic_bike.py, stats.py) not imported, but included as separate cells

In [ ]:
# from dynamic_bike import extract_df, merge_df, data_manip, save_excel
# from stats import df_avg

import pandas as pd
from pandas import ExcelWriter
import glob
import os
import sys
# import PySimpleGUI as sg

def start():
    event, values = sg.Window('Choice').Layout([[sg.Text('What would you like to do?')],
                                                [sg.Radio('Reorganize Raw Files', 'RADIO1', default=True)], #0
                                                [sg.Radio('Combine New Files', 'RADIO1')], #1
                                                [sg.Radio('Perform Basic Stats', 'RADIO1')],#2
                                                [sg.CloseButton('Submit')]]).Read()
    window = sg.Window('Bike Data Tool')
    window.Close()
    ## Define input and output folders
    if values[0] == True:
        event, values = sg.Window('Bike Data Tool').Layout([[sg.Text('Input folder')],
                                                            [sg.In(), sg.FolderBrowse()], #0
                                                            [sg.Text('Output folder')],
                                                            [sg.In(), sg.FolderBrowse()], #1
                                                            [sg.Text('Would you like to clean your data?')],      
                                                            [sg.Radio('Yes', "RADIO1", default=True), #2
                                                            sg.Radio('No', "RADIO1")],#3
                                                            [sg.CloseButton('Submit'), sg.CloseButton('Cancel')]]).Read()
        window = sg.Window('Bike Data Tool')    
        window.Close()
        if event == 'Submit':
            raw_folder = values[0]
            # raw_folder = 'C:\\Users\\albei\\OneDrive\\Desktop\\analyze\\input'
            output_folder=values[1]
            # output_folder = 'C:\\Users\\albei\\OneDrive\\Desktop\\analyze\\output'
            manip = values[2]
            user_input(raw_folder,output_folder,manip)
        else:
            quit
    elif values[2] == True:
        event, values = sg.Window('Bike Data Analysis Tool').Layout([[sg.Text('Input folder')],
                                                [sg.In(), sg.FolderBrowse()], #0
                                                [sg.Text('The files end in: ')],
                                                [sg.Radio('_new.xlsx', 'RADIO1', default=True), #1
                                                sg.Radio('_new_raw.xlsx', 'RADIO1')],
                                                [sg.Radio('Yes','RADIO2',default=True), #2
                                                sg.Radio('No','RADDIO2')], #3
                                                [sg.CloseButton('Submit'), sg.CloseButton('Cancel')]]).Read()
        window = sg.Window('Bike Data Analysis Tool')    
        window.Close()
        if event == 'Submit':
            output_folder = values[0]
            # raw_folder = 'C:\\Users\\albei\\OneDrive\\Desktop\\analyze\\input'

            # Progress bar window #
            manip = values[1]
            ent_yes = values[2]
            df_avg(output_folder, manip, ent_yes)
        else:
            quit
    elif values[1] == True:
        ## Define input and output folders
        event, values = sg.Window('Bike Data Combining Tool').Layout([[sg.Text('Which folder are the files in?')],
                                                            [sg.In(), sg.FolderBrowse()], #0
                                                            [sg.Text('Has the data been cleaned?')],      
                                                            [sg.Radio('Yes', "RADIO1", default=True), #1
                                                            sg.Radio('No', "RADIO1")],#2
                                                            [sg.CloseButton('Submit'), sg.CloseButton('Cancel')]]).Read()
        window = sg.Window('Bike Data Combining Tool')    
        window.Close()

        output_folder = values[0]
        manip = values[1]
        if event == 'Submit':
            combine_excels(manip, output_folder)
        if event == 'Cancel':
            raise SystemError(0)


## UI functions

NOTE: ```progressGUI()``` renamed to ```progressGUI_bike()```

In [4]:
def progressGUI_bike(raw_folder):
    count = len([name for name in os.listdir(raw_folder) if os.path.isfile(os.path.join(raw_folder, name))])
    layout = [[sg.Text('Press start to begin')],
            [sg.ProgressBar(count, orientation='h', size=(20, 20), key='progbar')],
            [sg.Output(size=(60,20))],
            [sg.Button('Start'),
            sg.Button('Quit')],
            ]
    window = sg.Window('Begin Restructuring and Cleaning', layout)
    return window

def user_input(raw_folder, output_folder, manip):     
    if manip == True:
    ### Info Gather GUI ###
        layout = [      
            [sg.Text('Replace all HR with "NaN" if they are <:', size=(30, 1),tooltip='Choosing 00 will disable this function'), sg.InputText()],      
            [sg.Text('Replace all HR with "NaN" if they are >:', size=(30, 1),tooltip='Choosing 00 will disable this function'), sg.InputText()],      
            [sg.Text('All rows will be deleted if the Cadence there is less than:', size=(30, 2), tooltip='Choosing 00 will disable this function'), sg.InputText()],
            [sg.Text('(Choosing 00 as the value will disable that option)',justification='center')],
            [sg.Submit(), sg.Cancel()]]      
        window = sg.Window('Data Cleaning', layout)  
        event, values = window.Read()   
        window.Close()
        if event == 'Submit':
            low_HR = int(values[0])
            high_HR = int(values[1])
            low_Cadence = int(values[2])
        ### Confirmation GUI still needed ###
            reorg_excels_and_manip(low_HR, high_HR, low_Cadence, manip, raw_folder, output_folder)
        else:
            raise SystemError(0)
    if manip == False:
        reorg_excels_no_manip(manip, raw_folder, output_folder)

## reorg functions

In [ ]:
def reorg_excels_and_manip(low_HR, high_HR, low_Cadence, manip, raw_folder, output_folder):
    neg_pow_df = pd.DataFrame()
    path = raw_folder  # where the raw csv files are located
    all_files = glob.glob(os.path.join(path, "*.csv"))  # make a list of paths
    
    # Progress bar window #
    i = 1
    window = progressGUI(raw_folder)
    #--   --#
    while True:
        event,values = window.Read()
        if event == 'Start' or None:
            for files in all_files:
                i = i+1
                try:
                    csv_file = os.path.splitext(os.path.basename(files))[
                        0]  # get file name without extension
                    ext_dic = extract_df(csv_file, raw_folder)
                    #-- Extract the dictionary into its variables --#
                    HR = ext_dic['HR']
                    Cadence = ext_dic['Cadence']
                    Power = ext_dic['Power']
                    Torque = ext_dic['Torque']
                    #--   --#
                    subject_data = merge_df(HR, Cadence, Power, Torque, csv_file)
                    manip_dic = data_manip(subject_data, csv_file, low_HR, high_HR, low_Cadence, manip, output_folder)

                    subject_data = manip_dic['subject_data']

                    perc_nan = manip_dic['perc_nan']
                    if perc_nan > 0:
                        print(perc_nan,"% of HR values have been replaced with 'NaN' in ", csv_file)
                    save_excel(subject_data, csv_file, manip, output_folder)

                    # Updates the progress bar #
                    event, values = window.Read(timeout=0)
                    if event == 'Cancel' or event is None:
                        break
                    window.Element('progbar').UpdateBar(i)
                    #--    --#
                    print(csv_file + ".csv reorganized!")
                except Exception as e:
                    csv_file = os.path.splitext(os.path.basename(files))[0]
                    print('ERROR WITH ',csv_file, '.csv: ', e)
                    window.Element('progbar').UpdateBar(i)
        
            finished(manip, output_folder, neg_pow_df)
        elif event == 'Quit':
            raise SystemError(0)

def reorg_excels_no_manip(manip, raw_folder, output_folder):
    neg_pow_df = pd.DataFrame()
    path = raw_folder  # where the raw csv files are located
    all_files = glob.glob(os.path.join(path, "*.csv"))  # make a list of paths
    
    # Progress bar window #
    i = 1
    window = progressGUI(raw_folder)
    #--   --#
    while True:
        event,values = window.Read()
        if event == 'Start' or None:
            for files in all_files:
                i = i+1
                try:
                    csv_file = os.path.splitext(os.path.basename(files))[0]  # get file name without extension
                    ext_dic = extract_df(csv_file,raw_folder)
                    #-- Extract the dictionary into its variables --#
                    HR = ext_dic['HR']
                    Cadence = ext_dic['Cadence']
                    Power = ext_dic['Power']
                    Torque = ext_dic['Torque']
                    #--   --#
                    subject_data = merge_df(HR, Cadence, Power, Torque, csv_file)
                    save_excel(subject_data, csv_file, manip, output_folder)

                    # Update the progress bar #
                    event, values = window.Read(timeout=0)
                    if event == 'Cancel' or event is None:
                        break
                    window.Element('progbar').UpdateBar(i)
                    #--   --#
                    print(csv_file + ".csv reorganized!")
                except Exception as e:
                    csv_file = os.path.splitext(os.path.basename(files))[0]
                    print('ERROR WITH ', csv_file, '.csv: ', e)
                    window.Element('progbar').UpdateBar(i)

            finished(manip, output_folder, neg_pow_df)
        elif event == 'Quit':
            raise SystemError(0)

## finished( )

NOTE: ```finished()``` renamed to ```finished_stats()```

In [7]:
def finished_bike(manip, output_folder, neg_pow_df):
    #### Finished Window with Options ###
    layout = [[sg.Text('Finished! What would you like to do next?')],
                [sg.Radio('Quit', "RADIO1", default=True),
                sg.Radio('Combine Files', "RADIO1"),
                sg.Radio ('Start Over', "RADIO1")],
                [sg.Submit()]]      
    window = sg.Window('File Restructuring Finished!', layout)    

    event, values = window.Read()    
    window.Close()

    if values[0] == True:
        raise SystemError(0)
    elif values[1] == True:
        combine_excels(manip, output_folder)
    elif values[2] == True:
        start()

## combine_excels( )

In [ ]:
def combine_excels(manip, output_folder):
    path = output_folder
    all_data = pd.DataFrame()
    #--- Convert Dataframe to Excel ---#   
    while True:
        if manip == True:
            for f in glob.glob(os.path.join(path, '*_new.xlsx')):
                try:
                    df = pd.read_excel(f)
                    all_data = all_data.append(df, ignore_index=True)
                except:
                    print('There is a problem with ',f)
            all_manip = output_folder + '/' + '[combined_data_manip].xlsx'
            writer = pd.ExcelWriter(all_manip)
            # save without name of columns and the row-numbers
            all_data.to_excel(writer, sheet_name='Sheet1', index=False)
            writer.save()
            ### Finished Notification GUI ###
            layout = [[sg.Text('Finished! Your new file is saved as [combined_data_manip].xlsx')],
                        [sg.Radio('Quit', "RADIO1", default=True, size=(10,1)), sg.Radio('Start Over', "RADIO1")],
                        [sg.Submit()]]      
            window = sg.Window('Finished!', layout)
            event, values = window.Read()    
            window.Close()
            
            again = values[0]
            if again == False:
                start()
            else:
                raise SystemError(0)
        if manip == False:
            for f in glob.glob(os.path.join(path, '*_new_raw.xlsx')):
                try:
                    df = pd.read_excel(f)
                    all_data = all_data.append(df, ignore_index=True)
                except:
                    print('There is a problem with ',f)
            all_manip = output_folder + '/' + '[combined_data_raw].xlsx'
            writer = pd.ExcelWriter(all_manip)
            # save without name of columns and the row-numbers
            all_data.to_excel(writer, sheet_name='Sheet1', index=False)
            writer.save()
            ### Finished Notification GUI ###
            layout = [[sg.Text('Finished! Your new file is saved as [combined_data_raw].xlsx')],
                        [sg.Radio('Quit', "RADIO1", default=True, size=(10,1)), sg.Radio('Start Over', "RADIO1")],
                        [sg.Submit()]]      
            window = sg.Window('Finished!', layout)
            event, values = window.Read()
            window.Close()
            
            again = values[0]
            if again == False:
                start()
            else:
                raise SystemError(0)

# start()

# dynamic_bike.py

## extract_df( )

In [ ]:
# dynamic_bike.py
import pandas as pd
import numpy as np
from pandas import ExcelWriter
#--- Manipulate an Excel File ---#

def extract_df(csv_file, raw_folder):
    csv_file2 = raw_folder + '/' + csv_file + '.csv'
    # import csv page. Sheetname refers to sheet in file
    sheet = pd.read_csv(csv_file2, delimiter=',')
    sheet.columns = ['Date', 'Time', 'Millitm', 'Marker', 'Tag', 'Status', 'Value'] # remove ; from ;Date
    tag = sheet.set_index(['Tag'])  # set Tag column as the default index    
    # NOTE: use isin to iterate: tag.loc['item one','item two']
    HR = tag.loc['{[CompactLogix]Heart_Rate}'].reset_index(drop=True) # select all rows that have this tag
    HR.rename(columns={'Value': 'HR'}, inplace=True)
    Cadence = tag.loc['{[CompactLogix]Rider_Cadence}'].reset_index(drop=True)
    Cadence.rename(columns={'Value': 'Cadence'}, inplace=True)
    Power = tag.loc['{[CompactLogix]Actual_Power}'].reset_index(drop=True)
    # rename Value column to Power
    Power.rename(columns={'Value': 'Power'}, inplace=True)
    Torque = tag.loc['{[CompactLogix]Actual_Torque}'].reset_index(drop=True)
    Torque.rename(columns={'Value': 'Torque'}, inplace=True)
    ext_dic = {
        'HR': HR,
        'Cadence': Cadence,
        'Power': Power,
        'Torque': Torque
    }
    return ext_dic


## merge_df( )

In [ ]:
def merge_df(HR, Cadence, Power, Torque, csv_file):
    #--- Merge Dataframes ---#
    subject_data = pd.merge(HR,
                            Cadence[['Time', 'Cadence']],
                            on='Time')
    subject_data = pd.merge(subject_data,  # left dataframe to merge to
                            # Right dataframe: select two columns
                            Power[['Time', 'Power']],
                            on='Time')  # merge based on matching "Time" column
    subject_data = pd.merge(subject_data,
                            Torque[['Time', 'Torque']],
                            on='Time')

    # subject_data.insert(0,'Name',csv_file)
    del subject_data['Torque']
    return subject_data

## data_manip( )

In [ ]:
def data_manip(subject_data, csv_file, low_HR, high_HR, low_Cadence, manip, output_folder):
    df = subject_data
    if low_Cadence != 00:
        df = df.drop(df[(df.Cadence <= low_Cadence)].index).reset_index(drop=True)

    ## Replace with NaN ##
    if low_HR !=00:
        df['HR'][df['HR'] <= low_HR] = np.nan
    if high_HR !=00:
        df['HR'][df['HR'] >= high_HR] = np.nan    
    ## ---------------- ##

    nans_made = df['HR'].isna().sum()
    total_rows = df.shape[0]
    perc_nan = round((nans_made/total_rows) * 100, 2)
    subject_data = df

    manip_dic = {
        'subject_data' : df,
        'perc_nan' : perc_nan
        }
    return manip_dic

## save_excel

In [ ]:
def save_excel(subject_data, csv_file, manip, output_folder):
    subject_data['ID'] = csv_file
    if manip == True:
        del subject_data['Time']
        del subject_data['Millitm']
        del subject_data['Status']
        del subject_data['Marker']
        csv_file = output_folder + '\\' + csv_file + '_new.xlsx'
        #--- Convert Dataframe to Excel ---#
        writer = ExcelWriter(csv_file)
        # , header=False) #save without name of columns and the row-numbers
        subject_data.to_excel(writer, sheet_name='Sheet1', index=False)
        writer.save()
    if manip == False:
        del subject_data['Status']
        del subject_data['Marker']
        csv_file = output_folder + '\\' + csv_file + '_new_raw.xlsx'
        #--- Convert Dataframe to Excel ---#
        writer = ExcelWriter(csv_file)
        # , header=False) #save without name of columns and the row-numbers
        subject_data.to_excel(writer, sheet_name='Sheet1', index=False)
        writer.save()    

# stats.py

## df_avg( )

In [ ]:
import pandas as pd
import numpy as np
import glob
import os
import nolds #sampen calc

from pandas import ExcelWriter
import PySimpleGUI as sg      


# Logic: create a dictionary of averages from file, then append that dictionary to df2. Do this for all files
# Columns are made from the keys, data from the values of the dictionary

def df_avg(output_folder, manip, ent_yes):
    if manip == True:
        file_list = glob.glob(os.path.join(output_folder, "*_new.xlsx"))  # make a list of paths
    elif manip == False:
        file_list = glob.glob(os.path.join(output_folder, "*_new_raw.xlsx"))
    df = []
    cols = ['ID', 'avg_HR', 'std_HR', 'SamEn_HR','avg_Cad', 'std_Cad', 'SamEn_Cad','avg_Pow', 'std_Pow', 'SamEn_Pow','neg_Pow', 'HR_NaN', 'Length']
    i = 1
    window = progressGUI(output_folder)
    while True:
        for x in file_list:
            i = i+1
            try:
                file = pd.read_excel(x)
                file_id = file.loc[0,'ID']

                means = round(np.mean(file),2) # Mean for each column
                stds = round(np.std(file),2)   # Standard deviation for each column
                file_length = file.shape[0]
                is_nan = file.isna().sum() # Number of NaN values for each column
                
                seconds = neg_Pow_count(file)

                sampEn_HR = round(nolds.sampen(file['HR']),2)
                sampEn_Cad = round(nolds.sampen(file['Cadence']),2)
                sampEn_Pow = round(nolds.sampen(file['Power']),2)

                df.append({'ID':file_id, 
                            'avg_HR':means['HR'],
                            'std_HR':stds['HR'],
                            'Sam_HR':sampEn_HR,
                            'avg_Cad':means['Cadence'],
                            'std_Cad':stds['Cadence'],
                            'Sam_Cad':sampEn_Cad, 
                            'avg_Pow':means['Power'],
                            'std_Pow':stds['Power'],
                            'Sam_Pow':sampEn_Pow,
                            'neg_Pow':seconds, 
                            'Length':file_length,
                            'HR_NaN':is_nan['HR'] 
                            #'Cad_NaN':is_nan['Cadence'],
                            #'Pow_NaN':is_nan['Power']
                            })

                #-- Updates the progress bar --#
                event, values = window.Read(timeout=0)
                print(file_id, ' analyzed')
                if event == 'Cancel' or event is None:
                    break
                window.Element('progbar').UpdateBar(i)
                
                #------------------------------#
            except Exception as e:
                print(file_id, ': ',e)
        
        df = pd.DataFrame(df,columns=cols)

        #-- Add Mean and Std to end of each columns --#
        df.loc['Mean'] = round(np.mean(df),2)
        df.loc['Std'] = round(np.std(df),2)
        df.loc['Mean','ID'] = 'Mean'
        df.loc['Std','ID'] = 'Std'
        #---------------------------------------------#
        avgs_save(df, output_folder, window,manip)
        window.Close()
    if ent_yes == True:
        layout = [[sg.Text('What was the resistance setting?')],
                [sg.Input()], # 0
                [sg.Text('What was the cadence setting?')],
                [sg.Input()], # 1
                [sg.Output(size=(60,20))],
                [sg.Button('Submit')]
                ]    
        window = sg.Window('Entropy Analysis Options!', layout)    
        event, values = window.Read()  

        res = values[0]
        cad = values[1]

        entropy_matlab(res,cad,output_folder)
    print('Finished!')

## entropy_matlab( )

In [1]:
def entropy_matlab(res, cad, root, output_folder):
    import matlab.engine as mat

    res = res # resistance setting used
    cad = cad # cadence setting used
    root = os.path.dirname(os.path.realpath(__file__)) # directory of matlab files

    eng = mat.start_matlab() # starts the matlab engine
    answer = eng.apsamen_cleaned(res, cad, root, output_folder, nargout=0) # calls the apsamen_cleaned.m file's function
    print(answer) # prints everything out, because of nargout=0 argument

## neg_Pow_count( )

In [ ]:
def neg_Pow_count(file):
    time = file[file.Power < 0].shape[0] # time on bike where Power is less than 0
    return time

## avgs_save( )

In [2]:
def avgs_save(df, output_folder, window,manip):
    if manip == True:
        print('Saving [basic_stats].xlsx file')
        event, values = window.Read(timeout=0)
        save_file = output_folder + '\\' + '[basic_stats].xlsx'
        writer = pd.ExcelWriter(save_file)
        df.to_excel(writer, sheet_name='Sheet1', index=False)
        writer.save()
        print('[basic_stats].xlsx saved')
    elif manip == False:
        print('Saving [basic_stats_raw].xlsx file')
        event, values = window.Read(timeout=0)
        save_file = output_folder + '\\' + '[basic_stats_raw].xlsx'
        writer = pd.ExcelWriter(save_file)
        df.to_excel(writer, sheet_name='Sheet1', index=False)
        writer.save()
        print('[basic_stats_raw].xlsx saved')
    finished(manip)

## finished( )

NOTE: ```finished()``` renamed to ```finished_stats()```

In [6]:
def finished_stats(manip):
    if manip == True:
        layout = [[sg.Popup('Finished! The file has been saved as [basic_stats].xlsx')],
                    [sg.Button('OK')]]      
        window = sg.Window('Statistical Analysis Finished!', layout)    
        event, values = window.Read()   
    elif manip == False:
        layout = [[sg.Popup('Finished! The file has been saved as [basic_stats_raw].xlsx')],
        [sg.Button('OK')]]      
        window = sg.Window('Statistical Analysis Finished!', layout)    
        event, values = window.Read()  
    if event == 'OK':
        raise SystemError(0)
    window.Close()

## progressGUI_stats

NOTE: ```progressGUI()``` renamed to ```progressGUI_stats()```

In [5]:
def progressGUI_stats(output_folder):
    count = len([name for name in os.listdir(output_folder) if os.path.isfile(os.path.join(output_folder, name))])
    layout = [[sg.Text('Press start to begin')],
            [sg.ProgressBar(count, orientation='h', size=(20, 20), key='progbar')],
            [sg.Output(size=(60,20))],
            [sg.Button('Start'),
            sg.Button('Quit')],
            ]
    window = sg.Window('Begin Restructuring and Cleaning', layout)
    return window